<a href="https://colab.research.google.com/github/itsfarhan21/Natural-Language-Processing-/blob/main/NaiveBayesAssignment3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [41]:
import pandas as pd
import numpy as np
import nltk
from nltk.corpus import words

In [42]:
vocabulary = {}
data = pd.read_csv("dataset_olshop.csv")
nltk.download("words")
set_words = set(words.words())

def build_vocabulary(current_mail):
    index = len(vocabulary)

    for word in current_mail:
        if word.lower() not in vocabulary and word.lower() not in set_words:
            vocabulary[word] = index
            index += 1

if __name__ == "__main__":
    for i in range(data.shape[0]):
        current_mail = data.iloc[i, 0].split()
        print(f'current mail is [i]/data.shape[0] and the \ length of vocabulary is {len(vocabulary)}')

        build_vocabulary(current_mail)

    file = open("vocabulary NLP.txt", "w")
    file.write(str(vocabulary))
    file.close()





[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Package words is already up-to-date!


current mail is [i]/data.shape[0] and the \ length of vocabulary is 0
current mail is [i]/data.shape[0] and the \ length of vocabulary is 3
current mail is [i]/data.shape[0] and the \ length of vocabulary is 6
current mail is [i]/data.shape[0] and the \ length of vocabulary is 37
current mail is [i]/data.shape[0] and the \ length of vocabulary is 40
current mail is [i]/data.shape[0] and the \ length of vocabulary is 46
current mail is [i]/data.shape[0] and the \ length of vocabulary is 47
current mail is [i]/data.shape[0] and the \ length of vocabulary is 56
current mail is [i]/data.shape[0] and the \ length of vocabulary is 61
current mail is [i]/data.shape[0] and the \ length of vocabulary is 66
current mail is [i]/data.shape[0] and the \ length of vocabulary is 94
current mail is [i]/data.shape[0] and the \ length of vocabulary is 99
current mail is [i]/data.shape[0] and the \ length of vocabulary is 129
current mail is [i]/data.shape[0] and the \ length of vocabulary is 131
current

In [43]:
import ast

data = pd.read_csv("dataset_olshop.csv")
file = open("vocabulary NLP.txt", "r")
contents = file.read()
vocabulary = ast.literal_eval(contents)

X = np.zeros((data.shape[0], len(vocabulary)))
y = np.zeros((data.shape[0]))

for i in range(data.shape[0]):
  mail = data.iloc[i, 0].split()

  for mail_words in mail:
    if mail_words in vocabulary :
      X[i, vocabulary[mail_words]] += 1
      y[i] = data.iloc[i,1]

np.save("dataX.npy", X)
np.save("dataY.npy", y)

In [44]:
class NaiveBayes():
  def __init__(self, X, y):
    self.num_examples, self.num_features = X.shape
    self.num_classes = len(np.unique(y))
    self.eps = 1e-6
  
  def fit(self, X, y):
    self.classes_mean = {}
    self.classes_variance = {}
    self.classes_prior = {}

    for s in range(self.num_classes):
      X_s = X[y==s]

      self.classes_mean[str(s)] = np.mean(X_s, axis = 0)
      self.classes_variance[str(s)] = np.var(X_s, axis=0)
      self.classes_prior[str(s)] = X_s.shape[0]/self.num_examples

  def predict(self, X):
    probs = np.zeros((self.num_examples, self.num_classes))

    for s in range(self.num_classes):
      prior = self.classes_prior[str(s)]
      probs_s = self.density_function(X, self.classes_mean[str(s)], self.classes_variance[str(s)])
      probs[:, s] = probs_s + np.log(prior)

    return np.argmax(probs, 1)

  def density_function(self, x, mean, sigma):
    const = -self.num_features/2 * np.log(2*np.pi) - 0.5*np.sum(np.log(sigma+self.eps))
    probs = 0.5*np.sum(np.power(x-mean, 2)/(sigma+self.eps), 1)
    return const - probs

if __name__ == "__main__":

  X = np.load("dataX.npy")
  y = np.load("dataY.npy")

  NB = NaiveBayes(X, y)
  NB.fit(X, y)
  y_pred = NB.predict(X)

  print(sum(y_pred == y)/X.shape[0])


0.9266666666666666
